<a href="https://colab.research.google.com/github/avk4714/DL_Projects/blob/master/Dropout_DNN_Cartpole.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Implementation of the Dropout Method to determine Cartpole dynamics**

In [0]:
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.autograd import Variable
from torch.distributions import Bernoulli
import matplotlib.pyplot as plt
import csv

In [0]:
class Net(nn.Module):
  def __init__(self, D_in1, n_hidden, D_out1, z_prob1, lam_mult1):
    super(Net, self).__init__()
    # Layers
    self.model_L_1 = torch.nn.Linear(D_in1, n_hidden, bias=True)
    self.model_D_1 = torch.nn.Dropout(p=0.5)
    self.model_L_2 = torch.nn.Linear(n_hidden, n_hidden, bias=True)
    self.model_D_2 = torch.nn.Dropout(p=0.5)
    self.model_L_3 = torch.nn.Linear(n_hidden, n_hidden, bias=True)
    self.model_D_3 = torch.nn.Dropout(p=0.5)
    self.model_L_4 = torch.nn.Linear(n_hidden, D_out1, bias=True)

  def forward(self, x):
    pred_1 = F.relu(self.model_D_1(self.model_L_1(x)))
    pred_2 = F.relu(self.model_D_2(self.model_L_2(pred_1)))
    pred_3 = F.relu(self.model_D_3(self.model_L_3(pred_2)))
    y_pred = self.model_L_4(pred_3)
    return y_pred

In [0]:
from numpy import genfromtxt

train_x = genfromtxt('train_x_data_2.csv', delimiter=',')
train_y = genfromtxt('train_y_data_2.csv', delimiter=',')
train_x_tnsr = torch.from_numpy(train_x)
train_y_tnsr = torch.from_numpy(train_y)

**Calling DNN for Model Training**

In [0]:
'''Training Parameter Settings'''
n_samples = 500
n_hidden = 500
z_prob = 0.5
lam_mult = 1e-2
d_in = 6            # Inputs are: [p, dp, dtheta, sin(theta), cos(theta), action]
d_out = 4           # Outputs are: [ddtheta, ddp, dtheta, dp]
runs = 10000

'''PyTorch Model Invocation'''
model = Net(d_in, n_hidden, d_out, z_prob, lam_mult)

'''Loss function and Optimizer'''
criterion = torch.nn.MSELoss(size_average=False)
optimizer = torch.optim.Adam(model.parameters(),lr=0.001,weight_decay=lam_mult)

'''Training runs'''
loss_data = []
for t in range(runs):
  pred_y_tnsr = model(train_x_tnsr.float())
  loss = criterion(pred_y_tnsr.float(), train_y_tnsr.float())
  if t % 100 == 99:
    loss_data.append([t, loss.item()])
    print(t, loss.item())
  
  # zero gradients, perform a backward pass and update the weights
  optimizer.zero_grad()
  loss.backward()
  optimizer.step()

/usr/local/lib/python3.6/dist-packages/torch/nn/_reduction.py:43: UserWarning: size_average and reduce args will be deprecated, please use reduction='sum' instead.
  warnings.warn(warning.format(ret))


99 2023.550048828125
199 1105.1629638671875
299 835.788818359375
399 674.0801391601562
499 562.4903564453125
599 498.0129089355469
699 449.7741394042969
799 427.79071044921875
899 414.48699951171875
999 380.3399353027344
1099 343.0301208496094
1199 325.7327575683594
1299 322.26910400390625
1399 312.6142883300781
1499 301.4775085449219
1599 289.80548095703125
1699 279.21142578125
1799 286.1510925292969
1899 272.9510803222656
1999 275.5283508300781
2099 263.2027587890625
2199 259.3119812011719
2299 258.2286071777344
2399 243.94532775878906
2499 246.82382202148438
2599 256.00445556640625


In [7]:
X = Variable(torch.randn(10, 6))
X

tensor([[ 0.3811, -1.8697,  0.7928, -0.8041, -0.2373, -0.8267],
        [ 0.1787,  0.2206,  0.9807,  0.3767, -0.5994,  0.9951],
        [-0.2172,  0.9065,  0.9084,  0.3914, -1.0015,  0.4489],
        [-2.4166, -0.0954, -1.4217,  0.6954,  0.7475,  0.0976],
        [-1.5799, -0.4525,  1.2509,  1.8448, -0.0173,  0.1357],
        [ 1.1951, -0.3800,  1.8139,  0.5576,  1.0930, -0.5879],
        [ 0.5069,  0.6907,  0.1328,  1.5611,  0.1822, -1.7584],
        [ 1.8869, -0.2115, -0.4195,  1.8641, -0.9411,  0.1731],
        [ 0.9840, -0.8503,  1.4222, -0.1173, -1.8323,  0.3421],
        [ 1.6001, -0.3717,  1.1233,  1.1001,  0.1368,  1.0194]])